In [ ]:
# Converting QTL json file to CSV format
import json
import csv

def json_to_csv(json_file, csv_file):
    with open(json_file) as f:
        data = json.load(f)
    with open(csv_file, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(data[0].keys())
        for row in data:
            writer.writerow(row.values())
    
json_path = "QTL_text.json"
csv_path = "QTL_text.csv"
json_to_csv(json_path, csv_path)

In [1]:
# Splitting into test and train dataframes
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("QTL_text.csv")

train, test = train_test_split(df, test_size=0.1, stratify=df["Category"])

train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)

In [114]:
!pip install "tensorflow<2.11" 

^C


  Using cached gast-0.4.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached pyasn1_modules-0.4.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
   ---------------------------------------- 0.0/455.9 MB ? eta -:--:--
   ---------------------------------------- 4.5/455.9 MB 26.9 MB/s eta 0:00:17
    --------------------------------------- 8.4/455.9 MB 22.6 MB/s eta 0:00:20
   - -------------------------------------- 12.6/455.9 MB 24.6 MB/s eta 0:00:18
   - -------------------------------------- 18.4/455.9 MB 23.2 MB/s eta 0:00:19
   -- ------------------------------------- 25.7/455.9 MB 25.1 MB/s eta 0:00:18
   -- ------------------------------------- 31.2/455.9 MB 25.7 MB/s eta 0:00:17
   --- ------------------------------------ 37.0/455.9 M

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires keras>=3.5.0, but you have keras 2.10.0 which is incompatible.
tensorflow-intel 2.18.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
tensorflow-intel 2.18.0 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.10.1 which is incompatible.


In [2]:
import pandas, numpy, string
import pandas as pd
import pickle
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, decomposition, ensemble, naive_bayes
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# import xgboost
# from keras import layers, models, optimizers
# from keras.preprocessing import text, sequence
import matplotlib.pyplot as plt

In [2]:
# read test_unlabeled.tsv and convert to csv
# test_df = pd.read_csv("test_unlabeled.tsv", sep='\t')
# test_df.to_csv("test_unlabeled.csv", index=False)

In [3]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("test.csv")
df_test = pd.read_csv("test_unlabeled.csv")

In [6]:
df_train['Category'].value_counts()

Category
0    9244
1     906
Name: count, dtype: int64

In [7]:
# Preprocessing Helper Functions

from nltk.stem.porter import PorterStemmer 
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import re, unidecode, string

# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('punkt_tab')

def remove_accented_chars(text):
    text = unidecode.unidecode(text)
    return text
def remove_numbers(text): 
    result = re.sub(r'\d+', '', text) 
    return result
def remove_slash_with_space(text): 
    return text.replace('\\', " ")
def remove_punctuation(text): 
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator) 
def text_lowercase(text): 
    return text.lower()     
def remove_whitespace(text): 
    return  " ".join(text.split()) 
def remove_stopwords(text): 
    stop_words = set(stopwords.words("english")) 
    word_tokens = word_tokenize(text) 
    filtered_text = [word for word in word_tokens if word not in stop_words] 
    return ' '.join(filtered_text)
def stem_words(text): 
    stemmer = PorterStemmer() 
    word_tokens = word_tokenize(text) 
    stems = [stemmer.stem(word) for word in word_tokens] 
    return ' '.join(stems)
def lemmatize_words(text): 
    lemmatizer = WordNetLemmatizer() 
    word_tokens = word_tokenize(text) 
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens] 
    return ' '.join(lemmas) 


In [8]:
# Perform preprocessing
def perform_preprocessing(text):
    text = remove_accented_chars(text)
    text = remove_numbers(text)
    text = remove_stopwords(text)
    text = text_lowercase(text)
    text = remove_slash_with_space(text)
    text = remove_punctuation(text)
    text = stem_words(text)
    text = lemmatize_words(text)
    text = remove_whitespace(text)
    return text

df_train['Title'] = df_train['Title'].apply(perform_preprocessing)
df_train['Abstract'] = df_train['Abstract'].apply(perform_preprocessing)
df_val['Title'] = df_val['Title'].apply(perform_preprocessing)
df_val['Abstract'] = df_val['Abstract'].apply(perform_preprocessing)
df_test['Title'] = df_test['Title'].apply(perform_preprocessing)
df_test['Abstract'] = df_test['Abstract'].apply(perform_preprocessing)

In [9]:
X_train = df_train['Abstract']
X_val = df_val['Abstract']
y_train = df_train['Category']
y_val = df_val['Category']
X_test = df_test['Abstract']

In [6]:
# Count vectors
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_train)
X_train_count =  count_vect.transform(X_train)
X_val_count =  count_vect.transform(X_val)

In [7]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(X_train)
X_train_tfidf =  tfidf_vect.transform(X_train)
X_val_tfidf =  tfidf_vect.transform(X_val)
X_test_tfidf =  tfidf_vect.transform(X_test)


In [10]:
# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,2), max_features=5000)
tfidf_vect_ngram.fit(X_train)
X_train_tfidf_ngram =  tfidf_vect_ngram.transform(X_train)
X_val_tfidf_ngram =  tfidf_vect_ngram.transform(X_val)
X_test_tfidf_ngram =  tfidf_vect_ngram.transform(X_test)

In [16]:
# Oversampling using Random Over Sampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=3)

X_train_tfidf_ngram_rand, y_train_rand = ros.fit_resample(X_train_tfidf_ngram, y_train)

print("Number of samples in each class in training set:")
print(y_train.value_counts())
print(y_train_rand.value_counts())

Number of samples in each class in training set:
Category
0    9244
1     906
Name: count, dtype: int64
Category
0    9244
1    9244
Name: count, dtype: int64


c:\Users\rohai\anaconda3\envs\new_env\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\rohai\anaconda3\envs\new_env\lib\site-packages\sklearn\base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
c:\Users\rohai\anaconda3\envs\new_env\lib\site-packages\sklearn\base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


In [9]:
# Oversampling using SMOTE

from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_train_tfidf_ngram_smoted, y_train_smoted = sm.fit_resample(X_train_tfidf_ngram, y_train)

# Print the number of samples in each class
print("Number of samples in each class in training set:")
print(y_train.value_counts())
print(y_train_smoted.value_counts())

Number of samples in each class in training set:
Category
0    9244
1     906
Name: count, dtype: int64
Category
0    9244
1    9244
Name: count, dtype: int64


c:\Users\rohai\anaconda3\envs\new_env\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [ ]:
# Logistic Regression on Ngram Level TF IDF Vectors
LRClassifier = linear_model.LogisticRegression()
LRClassifier.fit(X_train_tfidf_ngram, y_train)
predictions = LRClassifier.predict(X_val_tfidf_ngram)
acc = metrics.accuracy_score(predictions, y_val)
print("LR, Accuracy: ", acc)

# show me the unique values in prediction and their counts
unique, counts = numpy.unique(predictions, return_counts=True)
print(dict(zip(unique, counts)))

test_predictions = LRClassifier.predict(X_test_tfidf_ngram)

# Create a new dataframe with the PMID and its corresponding prediction
df_test['Category'] = test_predictions
df_test = df_test[['PMID', 'Category']]
df_test.to_csv("predictions.csv", index=False)

LR, Accuracy:  0.9618794326241135
{0: 1046, 1: 82}


In [ ]:
# Hyperparameter tuning for XGBoost classifier with hyperopt
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score

def hyperopt_train_test(params):
    xgbClassifier = xgboost.XGBClassifier(**params)
    return cross_val_score(xgbClassifier, X_train_tfidf_ngram_smoted.tocsc(), y_train_smoted, scoring='f1_weighted').mean()

space4xgb = {
    'learning_rate': hp.quniform('learning_rate', 0.01, 0.3, 0.01),
    'max_depth': hp.choice('max_depth', range(5, 30, 1)),
    'n_estimators': hp.choice('n_estimators', range(20, 205, 5)),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'subsample': hp.quniform('subsample', 0.1, 1, 0.01),
    'gamma': hp.quniform('gamma', 0, 0.5, 0.01),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.1, 1.0, 0.01),
    'scale_pos_weight': hp.quniform('scale_pos_weight', 1, 10, 1)
}

def f(params):
    acc = hyperopt_train_test(params)
    return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, space4xgb, algo=tpe.suggest, max_evals=100, trials=trials)
print('best:')
print(best)

100%|██████████| 100/100 [8:23:53<00:00, 302.33s/trial, best loss: -0.9684106080262396]  
best:
{'colsample_bytree': 0.49, 'gamma': 0.47000000000000003, 'learning_rate': 0.08, 'max_depth': 11, 'min_child_weight': 1.0, 'n_estimators': 4, 'scale_pos_weight': 1.0, 'subsample': 0.7000000000000001}


In [ ]:
# XGBoost on Ngram Level TF IDF Vectors

xgbClassifier = xgboost.XGBClassifier(colsample_bytree = 0.49, gamma = 0.47000000000000003, learning_rate=0.08, max_depth = 11, min_child_weight = 1, n_estimators=4, scale_pos_weight=1, subsample=0.7000000000000001)
xgbClassifier.fit(X_train_tfidf_ngram_smoted.tocsc(), y_train_smoted)

predictions = xgbClassifier.predict(X_val_tfidf_ngram.tocsc())
f1 = metrics.f1_score(predictions, y_val, average='weighted')
print("XGB, F1 Score: ", f1)

#print confusion matrix
print(metrics.confusion_matrix(y_val, predictions))

unique, counts = numpy.unique(predictions, return_counts=True)
print(dict(zip(unique, counts)))

test_predictions = xgbClassifier.predict(X_test_tfidf_ngram.tocsc())

# Create a new dataframe with the PMID and its corresponding prediction
df_test['Category'] = test_predictions
df_test = df_test[['PMID', 'Category']]
df_test.to_csv("predictions.csv", index=False)

XGB, F1 Score:  0.9203383569739952
[[966  61]
 [ 23  78]]
{0: 989, 1: 139}


In [22]:
# Multinomial Naive Bayes on Ngram Level TF IDF Vectors
NBClassifier = naive_bayes.MultinomialNB(alpha=0.1)
NBClassifier.fit(X_train_tfidf_ngram_smoted, y_train_smoted)
predictions = NBClassifier.predict(X_val_tfidf_ngram)
acc = metrics.accuracy_score(predictions, y_val)
print("NB, Accuracy: ", acc)

unique, counts = numpy.unique(predictions, return_counts=True)
print(dict(zip(unique, counts)))

print(metrics.confusion_matrix(y_val, predictions))

test_predictions = NBClassifier.predict(X_test_tfidf_ngram.tocsc())

# Create a new dataframe with the PMID and its corresponding prediction
df_test['Category'] = test_predictions
df_test = df_test[['PMID', 'Category']]
df_test.to_csv("predictions.csv", index=False)

NB, Accuracy:  0.8874113475177305
{0: 910, 1: 218}
[[905 122]
 [  5  96]]


In [11]:
# Hyperparameter Tuning for SVC

from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10], 'class_weight': [{0: 1, 1: 3}, {0: 1, 1: 5}, 'balanced']}
grid_search = GridSearchCV(svm.SVC(kernel='linear', probability=True), param_grid, scoring='recall')
grid_search.fit(X_train_tfidf_ngram_smoted, y_train_smoted)

print("Best Params:", grid_search.best_params_)

Best Params: {'C': 1, 'class_weight': {0: 1, 1: 3}}


In [14]:
# Support Vector Classifier on Ngram Level TF IDF Vectors
SVMClassifier = svm.SVC(C=1, kernel='linear', class_weight='balanced', probability=True)
SVMClassifier.fit(X_train_tfidf_ngram, y_train)
predictions = SVMClassifier.predict(X_val_tfidf_ngram)

acc = metrics.accuracy_score(predictions, y_val)
print("SVM, Accuracy: ", acc)

unique, counts = numpy.unique(predictions, return_counts=True)
print(dict(zip(unique, counts)))

print(metrics.confusion_matrix(y_val, predictions))


test_predictions = SVMClassifier.predict(X_test_tfidf_ngram)


# Create a new dataframe with the PMID and its corresponding prediction
df_test['Category'] = test_predictions

df_test['Category'].value_counts()
# df_test = df_test[['PMID', 'Category']]
# df_test.to_csv("predictions.csv", index=False)

SVM, Accuracy:  0.9547872340425532
{0: 992, 1: 136}
[[984  43]
 [  8  93]]


Category
0    1006
1      91
Name: count, dtype: int64

In [15]:
# predict with threshold
probs = SVMClassifier.predict_proba(X_test_tfidf_ngram)
threshold = 0.4
predictions = (probs[:,1] >= threshold).astype('int')

# test_predictions = SVMClassifier.predict(X_test_tfidf_ngram)


# Create a new dataframe with the PMID and its corresponding prediction
df_test['Category'] = predictions
df_test = df_test[['PMID', 'Category']]
df_test.to_csv("predictions.csv", index=False)

In [ ]:
# Hyperparameter tuning for KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

KNNClassifier = KNeighborsClassifier()
grid_search = GridSearchCV(KNNClassifier, param_grid, cv=5)
grid_search.fit(X_train_tfidf_ngram_smoted, y_train_smoted)
print(grid_search.best_params_)

{'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
0.9489405483199876
KNeighborsClassifier(metric='manhattan', n_neighbors=3, weights='distance')
{'mean_fit_time': array([0.01670041, 0.018787  , 0.01700034, 0.01810703, 0.01791053,
       0.02119756, 0.0191978 , 0.01990247, 0.01770821, 0.01820345,
       0.01890483, 0.01910219, 0.01799836, 0.019804  , 0.01920633,
       0.02031393]), 'std_fit_time': array([0.00231443, 0.00250998, 0.00309869, 0.00476054, 0.00430797,
       0.00665245, 0.0074157 , 0.0026519 , 0.00159385, 0.00133496,
       0.00294249, 0.00290431, 0.00189761, 0.00376387, 0.00417476,
       0.00130383]), 'mean_score_time': array([ 3.94989834,  3.92046518,  4.33785043,  4.31243343,  5.0637589 ,
        4.30111208,  4.18091264,  3.96510425, 12.91361585, 11.77229381,
       12.41022248, 11.68885608, 12.61834717, 11.454528  , 13.83981276,
       12.13556418]), 'std_score_time': array([0.16560931, 0.20581771, 0.13217201, 0.30105246, 0.459033  ,
       0.18274686, 0

In [10]:
# KNN Classifier on Ngram Level TF IDF Vectors
from sklearn.neighbors import KNeighborsClassifier

KNNClassifier = KNeighborsClassifier(n_neighbors=3, weights='distance', metric='manhattan')
KNNClassifier.fit(X_train_tfidf_ngram_smoted, y_train_smoted)
predictions = KNNClassifier.predict(X_val_tfidf_ngram)

acc = metrics.accuracy_score(predictions, y_val)
print("KNN, Accuracy: ", acc)

unique, counts = numpy.unique(predictions, return_counts=True)
print(dict(zip(unique, counts)))

test_predictions = KNNClassifier.predict(X_test_tfidf_ngram)

# Create a new dataframe with the PMID and its corresponding prediction
df_test['Category'] = test_predictions
df_test = df_test[['PMID', 'Category']]
df_test.to_csv("predictions.csv", index=False)

KNN, Accuracy:  0.9432624113475178
{0: 1075, 1: 53}


In [12]:
# Ensemble Classifier on Ngram Level TF IDF Vectors
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score

model = EasyEnsembleClassifier(n_estimators=10, sampling_strategy='not majority')
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X_train_tfidf_ngram, y_train, scoring='f1_weighted', cv=cv, n_jobs=-1, error_score='raise')
print('Mean F1: %.3f' % numpy.mean(n_scores))

model.fit(X_train_tfidf_ngram, y_train)
predictions = model.predict(X_val_tfidf_ngram)
f1 = metrics.f1_score(predictions, y_val, average='weighted')
print("EE, F1 Score: ", f1)

unique, counts = numpy.unique(predictions, return_counts=True)
print(dict(zip(unique, counts)))

print(metrics.confusion_matrix(y_val, predictions))

test_predictions = model.predict(X_test_tfidf_ngram)

# Create a new dataframe with the PMID and its corresponding prediction
df_test['Category'] = test_predictions
df_test = df_test[['PMID', 'Category']]
df_test.to_csv("predictions.csv", index=False)

Mean F1: 0.942


c:\Users\rohai\anaconda3\envs\new_env\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\rohai\anaconda3\envs\new_env\lib\site-packages\sklearn\base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
c:\Users\rohai\anaconda3\envs\new_env\lib\site-packages\sklearn\base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
c:\Users\rohai\anaconda3\envs\new_env\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(
c:\Users\rohai\anaconda3\envs\new_env\lib\site-packa

EE, F1 Score:  0.946611557317012
{0: 1039, 1: 89}
[[1002   25]
 [  37   64]]


In [45]:
# Random Forest Classifier on Ngram Level TF IDF Vectors
RFClassifier = ensemble.RandomForestClassifier(n_estimators=1000, class_weight='balanced_subsample')
RFClassifier.fit(X_train_tfidf_ngram, y_train)
predictions = RFClassifier.predict(X_val_tfidf_ngram)

f1 = metrics.f1_score(predictions, y_val, average='weighted')
print("RF, F1 Score: ", f1)

unique, counts = numpy.unique(predictions, return_counts=True)
print(dict(zip(unique, counts)))

print(metrics.confusion_matrix(y_val, predictions))

test_predictions = RFClassifier.predict(X_test_tfidf_ngram)

# Create a new dataframe with the PMID and its corresponding prediction
df_test['Category'] = test_predictions
df_test = df_test[['PMID', 'Category']]
df_test.to_csv("predictions.csv", index=False)

RF, F1 Score:  0.9540715811637186
{0: 1065, 1: 63}
[[1017   10]
 [  48   53]]


In [16]:
preds = pd.read_csv("predictions.csv")

unique, counts = numpy.unique(preds['Category'], return_counts=True)
print(dict(zip(unique, counts)))

{0: 1046, 1: 51}
